In [ ]:
from googleapiclient.discovery import build
import pandas as pd
pd.set_option('display.max_rows', None)

API_KEY = ""
CHANNEL= ""
youtube = build("youtube", "v3", developerKey=API_KEY)
num=80  # (num * 5)個動画が取得できる
    
def get_video_info(part,channelId,order,type,num):
    dic_list = []
    search_response = youtube.search().list(part=part,channelId=CHANNEL,order=order,type=type)
    output = youtube.search().list(part=part,channelId=channelId,order=order,type=type).execute()

    for i in range(num):
        dic_list = dic_list + output["items"]
        search_response = youtube.search().list_next(search_response,output)
        output = search_response.execute()

    df = pd.DataFrame(dic_list)
    return df
    #各動画毎に一意のvideoIdを取得

def get_statistics(id):
    statistics = youtube.videos().list(part = 'statistics', id = id).execute()
    return statistics['items'][0]['statistics']

#searchでとってきた動画のsnippet
df = get_video_info(part='snippet',channelId=CHANNEL,order='viewCount',type='video',num=num)
#dfからvideoIdをまとめたデータ
df1 = pd.DataFrame(list(df["id"]))["videoId"]
#dfからpublishedAtとtitleをまとめたデータ
df2 = pd.DataFrame(list(df['snippet']))[["publishedAt",'title']]
#df1とdf2を結合
ddf = pd.concat([df1,df2], axis = 1)
#df1のidを使って動画のstatisticsを集めたデータ
df_static = pd.DataFrame(list(df1.apply(lambda x : get_statistics(x))))

df_output = pd.concat([ddf,df_static], axis = 1)

df_output


参考

https://qiita.com/g-k/items/7c98efe21257afac70e9

In [ ]:
df_output.to_csv("xxx.csv")